# **Settings**

# 라이브러리 불러오기

In [2]:
import os
import sys
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import TextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
import pandas as pd
import networkx as nx
import math
import ast
import torch
# os.chdir('/Users/mac/AIworkspace/LLMWORKSPACE/RAG_Rec')
# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# 데이터 불러오기

In [14]:
print(os.getcwd())
device = torch.device("mps")

/Users/mac/AIworkspace/LLMWORKSPACE/RAG_Rec


In [3]:
# file_path = ('data/movie_record_per_user.csv')
file_path='data/train_movie.csv'
data = pd.read_csv(file_path)
# 'movie_explain' 열을 리스트로 변환
data['movie_explain'] = data['movie_explain'].apply(ast.literal_eval)

In [6]:
data

,UserId,movie_explain
0,1,"[3186 (Drama) ratings: 4, 1270 (Comedy|Sci-Fi)..."
1,2,"[1198 (Action|Adventure) ratings: 4, 1210 (Act..."
2,3,"[593 (Drama|Thriller) ratings: 3, 2858 (Comedy..."
3,4,[1210 (Action|Adventure|Romance|Sci-Fi|War) ra...
4,5,"[2717 (Comedy|Horror) ratings: 1, 908 (Drama|T..."
...,...,...
6035,6036,"[1721 (Drama|Romance) ratings: 4, 2428 (Horror..."
6036,6037,"[1882 (Action|Sci-Fi) ratings: 1, 3508 (Wester..."
6037,6038,"[920 (Drama|Romance|War) ratings: 3, 3396 (Chi..."
6038,6039,"[282 (Drama) ratings: 4, 111 (Drama|Thriller) ..."


## 1. 청킹 + 헤더 사용 
- 평점과 장르가 갑자기 변화하는 구간을 포착하여 해당 구간을 기준으로 청킹 
- 청크의 수가 너무 적으면 안 되므로 최소 10개가 되도록 
- 이후 헤더를 활용하여 한 사용자의 모든 청크에 요약을 하여 변화를 파악하도록 함 


In [7]:
import pandas as pd
import ast
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def chunk_interactions_with_minimum(interactions, rating_threshold=2, min_items=10):
    """
    평점 변화 및 장르 변화 기준으로 청킹 후, 최소 10개의 상호작용이 포함되도록 조정.

    Args:
        interactions (list): 사용자 상호작용 리스트.
        rating_threshold (int): 평점 변화 임계값.
        min_items (int): 최소 상호작용 개수.

    Returns:
        list: 최소 상호작용 개수를 만족하는 청킹된 리스트.
    """
    chunks = []  # 청킹 결과 저장
    current_chunk = [interactions[0]]  # 첫 번째 상호작용으로 초기화

    for i in range(1, len(interactions)):
        prev = interactions[i - 1]
        current = interactions[i]

        # 이전 및 현재 상호작용의 평점과 장르 추출
        prev_rating = int(prev.split('ratings: ')[1])
        current_rating = int(current.split('ratings: ')[1])
        prev_genres = set(prev.split(' (')[1].split(')')[0].split('|'))
        current_genres = set(current.split(' (')[1].split(')')[0].split('|'))

        # 조건 1: 장르가 포함되지 않은 경우
        genre_disjoint = len(prev_genres & current_genres) == 0

        # 조건 2: 평점 변화가 임계값 이상인 경우
        rating_diff = abs(current_rating - prev_rating) >= rating_threshold

        # 조건을 만족하면 청킹
        if genre_disjoint or rating_diff:
            if len(current_chunk) >= min_items:
                chunks.append(current_chunk)  # 현재 청크 저장
                current_chunk = []  # 새로운 청크 시작

        current_chunk.append(current)

    # 마지막 청크 추가
    if current_chunk:
        chunks.append(current_chunk)

    # 최소 상호작용 개수 보장
    if len(chunks) > 1:
        merged_chunks = []
        temp_chunk = []

        for chunk in chunks:
            temp_chunk.extend(chunk)
            if len(temp_chunk) >= min_items:
                merged_chunks.append(temp_chunk)
                temp_chunk = []

        if temp_chunk:  # 남아 있는 항목 처리
            if merged_chunks:
                merged_chunks[-1].extend(temp_chunk)
            else:
                merged_chunks.append(temp_chunk)

        return merged_chunks

    return chunks  # 청킹 결과 반환

- 최소보장이 없는 코드 
- 최소보장이 없는 경우, 청크의 길이가 극단적인 경우가 발생하므로 유의미한 청크 검색이 불가능

In [9]:
# # 최소보장 없는 코드 
# def chunk_interactions(interactions, rating_threshold=2):
#     chunks = []  # 청킹 결과 저장
#     current_chunk = [interactions[0]]  # 첫 번째 상호작용으로 초기화

#     for i in range(1, len(interactions)):
#         prev = interactions[i - 1]
#         current = interactions[i]

#         # 이전 및 현재 상호작용의 평점과 장르 추출
#         prev_rating = int(prev.split('ratings: ')[1])
#         current_rating = int(current.split('ratings: ')[1])
#         prev_genres = set(prev.split(' (')[1].split(')')[0].split('|'))
#         current_genres = set(current.split(' (')[1].split(')')[0].split('|'))

#         # 조건 1: 장르가 포함되지 않은 경우
#         genre_disjoint = len(prev_genres & current_genres) == 0

#         # 조건 2: 평점 변화가 임계값 이상인 경우
#         rating_diff = abs(current_rating - prev_rating) >= rating_threshold

#         # 조건을 만족하면 청킹
#         if genre_disjoint or rating_diff:
#             chunks.append(current_chunk)  # 현재 청크 저장
#             current_chunk = []  # 새로운 청크 시작

#         current_chunk.append(current)

#     # 마지막 청크 추가
#     if current_chunk:
#         chunks.append(current_chunk)

#     return chunks  # 청킹 결과 반환

In [8]:

all_chunks = []  # 모든 청크를 저장할 리스트
metadata_list = []  # 메타데이터 저장 리스트
chunked_by_user = {}  # 사용자별 청킹 결과 저장

# 사용자별로 청킹 수행
for _, row in data.iterrows():
    user_id = row['UserId']  # 사용자 ID
    interactions = row['movie_explain']  # 상호작용 리스트

    # 청킹 수행
    chunks = chunk_interactions_with_minimum(interactions, rating_threshold=2, min_items=5)
    # chunks=chunk_interactions(interactions, rating_threshold=3)
    # 사용자별로 청크 저장
    chunked_by_user[user_id] = chunks

    # 각 청크에 대해 메타데이터 추가
    for index, chunk in enumerate(chunks):
        chunk_text = " ".join(chunk)  # 리스트를 문자열로 변환
        all_chunks.append(chunk_text)
        metadata_list.append({"UserId": user_id, "chunk_index": index})  # 사용자 ID와 청크 번호 추가


In [9]:
len(all_chunks)

135399

# 벡터 스토어 불러오기 
- header 존재
1. vectorstore_index_full_movie_header
2. vectorstore_index_header_ratings_min10
- header 없음
3. vectorstore_index_ratings_min10, vectorstore_index_ratings_min5_no ,vectorstore_index_ratings_no_limit => no-title
4. vectorstore_index_full_movie_nochunking => no-chunking, no-title
5. vectorstore_index_ratings_min10_with_title
6. vectorstore_index_full_movie_nochunking_with_title



In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(
    "vectorstore_index_ratings_min5_no",
    embeddings,
    allow_dangerous_deserialization=True  # 역직렬화 허용
)

/tmp/ipykernel_372884/321042087.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/rlaalsduf/miniconda3/envs/rag_rec/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 정답 데이터셋 

In [15]:
file_path_test='data/test_movie.csv'
# 최신 구매 기록을 가져옴
purchase_history=data.iloc[0]['movie_explain']
# 정답 데이터셋 가져옴
df_test=pd.read_csv(file_path_test)
# 정답
df_test.iloc[0].movie_explain


'["48 (Animation|Children\'s|Musical|Romance) ratings: 5"]'

In [16]:
# purchase history(최신 구매기록)을 본 타 사용자들의 구매기록과 비교하여 검색
query = " ".join(purchase_history[-1:])  # 리스트를 문자열로 변환
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
records = retriever.get_relevant_documents(query)
print(query)
records

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_1921/2461333566.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  records = retriever.get_relevant_documents(query)


1907 (Animation|Children's) ratings: 4


[Document(metadata={'UserId': 264, 'chunk_index': 59}, page_content="1907 (Animation|Children's) ratings: 3 1033 (Animation|Children's) ratings: 2 2014 (Children's|Comedy) ratings: 3 2047 (Children's) ratings: 3 3287 (Animation|Children's) ratings: 2"),
 Document(metadata={'UserId': 5605, 'chunk_index': 17}, page_content="1907 (Animation|Children's) ratings: 2 1025 (Animation|Children's) ratings: 4 2085 (Animation|Children's) ratings: 4 2090 (Animation|Children's) ratings: 2 2137 (Animation|Children's) ratings: 2 2355 (Animation|Children's|Comedy) ratings: 3"),
 Document(metadata={'UserId': 5664, 'chunk_index': 15}, page_content="1907 (Animation|Children's) ratings: 2 1025 (Animation|Children's) ratings: 4 2048 (Animation|Children's) ratings: 4 2092 (Animation|Children's|Musical) ratings: 2 3054 (Animation|Children's) ratings: 4"),
 Document(metadata={'UserId': 4523, 'chunk_index': 7}, page_content="1907 (Animation|Children's) ratings: 1 62 (Drama) ratings: 4 1721 (Drama|Romance) ratin

# 메타 데이터를 활용하여 검색한 정보 앞뒤의 내용을 함께 가져옴 
- window size 설정
- 검색 개수 k 설정

In [18]:
from typing import List
from langchain.vectorstores import FAISS
from langchain.schema import Document

def get_documents_with_context(
    vectorstore: FAISS,
    query: str,
    k: int = 10,
    context_window: int = 1
) -> List[List[Document]]:
    """
    FAISS 벡터스토어에서 검색 결과와 각 결과의 앞뒤 문서들을 함께 반환합니다.
    
    Args:
        vectorstore: FAISS 벡터스토어 인스턴스
        query: 검색 쿼리
        k: 검색할 문서 수 (default: 10)
        context_window: 앞뒤로 가져올 문서 수 (default: 1)
    
    Returns:
        List[List[Document]]: 각 검색 결과에 대해 [이전 문서들, 현재 문서, 다음 문서들]을 포함하는 리스트
    """
    # 1. 초기 검색 수행
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    results = retriever.get_relevant_documents(query)
    
    # 2. 모든 문서와 메타데이터를 딕셔너리로 구성
    # UserId를 키로 하고, 그 안에 chunk_index를 키로 하는 중첩 딕셔너리 생성
    all_docs = {}
    for doc_id, doc in enumerate(vectorstore.docstore._dict.values()):
        user_id = doc.metadata['UserId']
        chunk_idx = doc.metadata['chunk_index']
        
        if user_id not in all_docs:
            all_docs[user_id] = {}
        all_docs[user_id][chunk_idx] = doc
    
    context_results = []
    
    # 3. 각 검색 결과에 대해 컨텍스트 추가
    for doc in results:
        current_user_id = doc.metadata['UserId']
        current_chunk_index = doc.metadata['chunk_index']
        
        context_docs = []
        
        # 이전 문서들 추가
        for i in range(current_chunk_index - context_window, current_chunk_index):
            if current_user_id in all_docs and i in all_docs[current_user_id]:
                context_docs.append(all_docs[current_user_id][i])
        
        # 현재 문서 추가
        context_docs.append(doc)
        
        # 다음 문서들 추가
        for i in range(current_chunk_index + 1, current_chunk_index + context_window + 1):
            if current_user_id in all_docs and i in all_docs[current_user_id]:
                context_docs.append(all_docs[current_user_id][i])
        
        context_results.append(context_docs)
    
    return context_results


In [19]:
context_results = get_documents_with_context(vectorstore, query, k=10, context_window=1)
# 중첩된 리스트 평탄화
flattened_results = [doc for sublist in context_results for doc in sublist]
record_summary = "\n".join([doc.page_content for doc in flattened_results])
record_summary

"3438 (Action|Children's|Fantasy) ratings: 1 3114 (Animation|Children's|Comedy) ratings: 3 3066 (War) ratings: 3 1101 (Action|Romance) ratings: 5 3253 (Comedy) ratings: 2\n1907 (Animation|Children's) ratings: 3 1033 (Animation|Children's) ratings: 2 2014 (Children's|Comedy) ratings: 3 2047 (Children's) ratings: 3 3287 (Animation|Children's) ratings: 2\n1292 (Comedy) ratings: 4 2921 (Western) ratings: 3 3623 (Action|Thriller) ratings: 3 3510 (Drama|Thriller) ratings: 4 3505 (Thriller) ratings: 4\n594 (Animation|Children's|Musical) ratings: 3 2876 (Animation|Children's) ratings: 2 588 (Animation|Children's|Comedy|Musical) ratings: 4 595 (Animation|Children's|Musical) ratings: 4 2096 (Animation|Children's|Musical) ratings: 4 2495 (Animation|Sci-Fi) ratings: 5\n1907 (Animation|Children's) ratings: 2 1025 (Animation|Children's) ratings: 4 2085 (Animation|Children's) ratings: 4 2090 (Animation|Children's) ratings: 2 2137 (Animation|Children's) ratings: 2 2355 (Animation|Children's|Comedy) ra

# 그래프 구조를 활용하여 검색된 사람들의 교집합 정리 

In [20]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import re

def get_user_movies(data):
    user_movies = defaultdict(list)

    for doc in data:
        user_id = doc.metadata['UserId']
        page_content = doc.page_content

        # 정규식을 사용하여 영화 ID 추출 (ratings 이전 내용만)
        movie_ids = re.findall(r'(\d+)(?= \()', page_content)

        # 사용자별로 영화 ID 추가
        user_movies[user_id].extend(movie_ids)

    return user_movies

def create_user_movie_graph(user_movies):
    # 그래프 생성
    G = nx.Graph()

    # 사용자와 영화 노드 추가 및 엣지 생성
    for user_id, movies in user_movies.items():
        user_node = f"User {user_id}"
        G.add_node(user_node, type='user')

        for movie_id in movies:
            movie_node = f"Movie {movie_id}"
            G.add_node(movie_node, type='movie')
            G.add_edge(user_node, movie_node)

    return G

def visualize_graph(G):
    # 그래프 시각화
    plt.figure(figsize=(20, 15))

    # 사용자와 영화 노드 분리
    users = [node for node in G.nodes() if G.nodes[node]['type'] == 'user']
    movies = [node for node in G.nodes() if G.nodes[node]['type'] == 'movie']

    # 레이아웃 설정
    pos = nx.spring_layout(G, k=0.5, iterations=50)

    # 노드 그리기
    nx.draw_networkx_nodes(G, pos, nodelist=users, node_color='lightblue', node_size=300, alpha=0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=movies, node_color='lightgreen', node_size=200, alpha=0.8)

    # 엣지 그리기
    nx.draw_networkx_edges(G, pos, alpha=0.5)

    # 레이블 그리기
    nx.draw_networkx_labels(G, pos, font_size=8, font_weight="bold")

    plt.title("User-Movie Relationship Graph", fontsize=16)
    plt.axis('off')
    plt.tight_layout()

    # 그래프 저장
    plt.savefig('user_movie_graph.png', dpi=300, bbox_inches='tight')
    plt.close()

import re

# 과거 기록에서 영화 ID 추출
def extract_previous_movie_ids(purchase_history):
    # 영화 ID만 추출 (숫자와 괄호 전까지만 가져옴)
    previous_movies = re.findall(r'(\d+)(?=\s\()', ' '.join(purchase_history))
    return set(previous_movies)

# 동시 시청 영화에서 이전 기록을 제외하는 함수
def filter_movies_by_history(top_movies, previous_movie_ids):
    # 영화 ID만 추출하여 비교 후 제외
    filtered_movies = [(movie, count) for movie, count in top_movies if movie.split()[1] not in previous_movie_ids]
    return filtered_movies
def get_top_10_common_movies(G):
    # 영화 노드만 필터링
    movies = [node for node in G.nodes() if G.nodes[node]['type'] == 'movie']

    # 영화별 연결된 사용자 수 계산
    movie_view_counts = {movie: len(list(G.neighbors(movie))) for movie in movies}

    # 사용자 수 기준으로 내림차순 정렬하여 상위 10개 추출
    top_10_movies = sorted(movie_view_counts.items(), key=lambda x: x[1], reverse=True)[:20]

    # 결과 반환
    return top_10_movies

# 사용자별 영화 ID 가져오기
user_movies = get_user_movies(flattened_results)

# 사용자-영화 그래프 생성
G = create_user_movie_graph(user_movies)

previous_movie_ids = extract_previous_movie_ids(purchase_history)

# Top-10 영화 리스트 가져오기
top_movies = get_top_10_common_movies(G)

# 이전 기록에 포함되지 않은 영화만 필터링
filtered_movies = filter_movies_by_history(top_movies, previous_movie_ids)

# 결과 출력
for movie, count in filtered_movies:
    print(f"{movie}: watched by {count} users")



Movie 2090: watched by 4 users
Movie 1025: watched by 3 users
Movie 2137: watched by 3 users
Movie 2081: watched by 3 users
Movie 48: watched by 3 users
Movie 3253: watched by 2 users
Movie 1033: watched by 2 users
Movie 2876: watched by 2 users
Movie 2096: watched by 2 users
Movie 2085: watched by 2 users
Movie 2078: watched by 2 users
Movie 3034: watched by 2 users
Movie 2092: watched by 2 users
Movie 1265: watched by 2 users


- gpt-4o-mini는 128k 토큰 입력을 받을 수 있다.

### test

In [22]:
import re
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate



# 결과를 저장할 리스트
results = []

# 1부터 30까지 반복
for idx in range(0, 6040):
    # 정답 추출
    test_answer = df_test.iloc[idx]['movie_explain']

    # movie_name 추출
    try:
        movie_name = re.search(r"\d+", test_answer).group()
    except AttributeError:
        movie_name = None  # 매칭 실패 시 None으로 처리

    ##-------------------------------------------------------------

    purchase_history = data.iloc[idx]['movie_explain']
    query = " ".join(purchase_history[-1:])

    context_results = get_documents_with_context(vectorstore, query, k=10, context_window=1)
    # 중첩된 리스트 평탄화
    flattened_results = [doc for sublist in context_results for doc in sublist]
    user_movies = get_user_movies(flattened_results)
    G = create_user_movie_graph(user_movies)
    previous_movie_ids = extract_previous_movie_ids(purchase_history)
    top_movies = get_top_10_common_movies(G)
    filtered_movies = filter_movies_by_history(top_movies, previous_movie_ids)

    # filtered_movies를 프롬프트 형식에 맞게 변환
    record_summary = "\n".join([
        f"{movie}: watched by {count} users"
        for movie, count in filtered_movies[:10]
    ])

    # LLM 프롬프트 생성
    template = """
Based on the following frequently watched movies:
{record_summary}

Recommend 10 movies for the user. Prioritize movies that:
1. Appear frequently in the given record summary.
2. Match user preferences based on similar viewers.

Provide recommendations as:
- Movie ID
- Movie ID
...
- Movie ID
"""

    prompt = ChatPromptTemplate.from_template(template)

    # LLM 연결 및 실행
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    chain = prompt | llm | StrOutputParser()

    try:
        response = chain.invoke({"record_summary": record_summary})

        # movie_title 추출
        movie_titles = re.findall(r"(\d+)", response)
    except Exception as e:
        movie_titles = []  # 오류 발생 시 빈 리스트 처리

    # movie_name이 movie_titles에 포함되는지 여부
    is_included = movie_name in movie_titles if movie_name else False

    # 결과 저장
    results.append({
        "movie_title": movie_titles,
        "movie_name": movie_name,
        "is_included": is_included


    })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)


In [23]:
results_df

,movie_title,movie_name,is_included
0,"[10, 2090, 1025, 2137, 2081, 48, 3253, 1033, 2...",48,True
1,"[10, 1779, 1391, 173, 1917, 1037, 196, 1831, 2...",1917,True
2,"[10, 3861, 3869, 3948, 3566, 3618, 3785, 3617,...",2081,False
3,"[10, 110, 2571, 1954, 1258, 1617, 1704, 1197, ...",1954,True
4,"[10, 832, 1094, 597, 2694, 3717, 70, 2325, 585...",288,False
...,...,...,...
6035,"[10, 3662, 2717, 2163, 879, 3476, 2003, 2367, ...",2643,False
6036,"[10, 1345, 1258, 1278, 2716, 1350, 2454, 3476,...",435,False
6037,"[10, 223, 3052, 2997, 2858, 2396, 3253, 1517, ...",1183,False
6038,"[10, 1254, 908, 527, 969, 1207, 1221, 2028, 11...",1254,True


## Evaluation

In [27]:
results_df.is_included.value_counts()
results_df.to_csv('first_ex_meta_enrich_graph.csv',index=False)

### HIT@5

In [26]:
df =results_df

# Hit@k 계산 (k=5)
k = 10
df['hit'] = df.apply(lambda row: row['movie_name'] in row['movie_title'][:k], axis=1)

# 전체 Hit@k 계산
hit_at_k = df['hit'].mean()  # Hit@k 비율
print(f"Hit@{k}: {hit_at_k}")


Hit@10: 0.08509933774834437
